In [ ]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:
%pylab inline

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from PIL import Image
import datetime 
import time
from ipywidgets import Button, Layout
import numpy as np
import sklearn
import time
from ipywidgets import Button, Layout

In [ ]:
clear_output()
# load=  widgets.IntProgress(
#     value=0,
#     min=0,
#     max=3,
#     step=1,
#     description='Loading Data:',
#     bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
#     orientation='horizontal'
# )
# ldvbox  = widgets.VBox([load])
# loading = widgets.HBox([ldvbox])
# display(loading)
ES_ = pd.read_pickle('ES.bugssData')
#load.value = 1
EB_ = pd.read_pickle('EB.bugssData')
#load.value = 2
W_ = pd.read_pickle('W.bugssData')
#load.value = 3
#ES.drop(ES.index)
ES_['datetime']=ES_['DateTime']
EB_['datetime']=EB_['DateTime']
W_['datetime']=W_['DateTime']

ES=ES_.set_index('datetime')
EB=EB_.set_index('datetime')
E=W_.set_index('datetime')

In [ ]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix
# !jupyter serverextension enable voila --debug

In [ ]:
def CleanList(LST,removeCommas,removeMore):
    listStr = str(LST)
    listStr = listStr.replace("'",'')
    if removeCommas==True:
        #listStr = listStr.replace(",,",',')
        listStr = listStr.replace(",, ",', ')
    listStr = listStr.replace('\\','')
    listStr = listStr.replace('/','')
    
    String = listStr[1:(len(listStr)-(1+removeMore))]
    #print(String)
    return(String)

In [ ]:
def DPlot(Datasets,DataPoints,Scale,StartDate,EndDate,figx,figy,Style):
    prg1 = len(Datasets)
    prg2 = len(DataPoints)
    prg3 = prg1*prg2
    prg = 0
    if StartDate == None:
        txtStartDate = '2018-1-1'
    else:
        txtStartDate = StartDate.strftime('%Y-%m-%d')
    if EndDate == None:
        txtEndDate = '2020-12-31'
    else:
        txtEndDate = EndDate.strftime('%Y-%m-%d')
#     cut1=0
#     cut2=0
#     if prg1 ==1:
#         cut1 = 1
#     if prg2 ==1:
#         cut2 = 1
    Filename = (CleanList(Datasets,False,1-(prg1-1)^0)+' - '+
                CleanList(DataPoints,False,1-(prg2-1)^0)+' - from '+
                txtStartDate+' to '+txtEndDate)
    extension = ('output/'+ Filename+'.png')
    print('"PATH/'+extension+'"')
    progress.max = prg3+1
    figsize(figx,figy)
    ax = plt.gca()
    plt.style.use(Style)
    if Scale == 2:
        plt.yscale('log')
    for ds in Datasets:
        if ds == 'East Surface':
            df=ES.loc[StartDate:EndDate]
        elif ds == 'East Bottom':
            df=EB.loc[StartDate:EndDate]
        else:
            df=W.loc[StartDate:EndDate]
        for dp in DataPoints:
            df.plot('DateTime',dp,ax=ax, label = (ds, dp))
            prg+=1
            progress.value = prg
    plt.savefig(extension)
    img = Image.open(extension)
    img.show()
    progress.value = progress.max
    time.sleep(.1)
    progress.value = 0


In [ ]:
def TrimYR(dataset,year,months,DateColumn,Name):
    m1 = months[0].month
    m2 = months[1].month
    From = str(year)+'-'+str(m1)
    To = str(year)+'-'+str(m2)
    DStrim = dataset.loc[From:To]
    DStrim['Name'] = Name
    return DStrim

def YPlot(datasets,DataPoints,Scale,Years,Months,figx,figy,Style):
    Dsets = []
    prg1 = len(datasets)*len(Years)
    prg2 = len(DataPoints)
    prg3 = prg1*prg2+prg1
    prg = 0
    progress.max = prg3+1
    figsize(figx,figy)
    ax = plt.gca()
    plt.style.use(Style)
    if Scale == 2:
        plt.yscale('log')
    for ds in datasets:
        if ds == 'East Surface':
            df=ES
        elif ds == 'East Bottom':
            df=EB
        else:
            df=W
        for yr in Years:
            Dsets.append(TrimYR(df,yr,Months,'DateTime',ds+'-'+yr))
            prg+=1
            progress.value = prg
    for ds in Dsets:
        DF = pd.DataFrame(ds)
        DF['DTime']=DF['DateTime'].dt.strftime('%m-%d %X')
        for dp in DataPoints:
            DF.plot('DTime',dp,ax=ax, label = (DF['Name'][0]+'-'+dp))
            prg+=1
            progress.value = prg
            

In [ ]:
envDatapoints = [('Salinity_ppt','Salinity_ppt'), ('pH','pH'),
              ('DO_mg/L','DO_mg/L'), ('Turb_NTU','Turb_NTU'),
              ('Chl_ug/L','Chl_ug/L'),('Temp_C','Temp_C')]

############# GENERAL #####################################################
Header = widgets.HTML(value='''<h1>Dαtα Ploting<h1>''',color='lime',font_size=75)
figx=widgets.IntSlider(
    value=10,
    min=5,
    max=40,
    step=1,
    description='X axis size:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
figx.style.handle_color="lawngreen"

figy=widgets.IntSlider(
    value=10,
    min=5,
    max=20,
    step=-1,
    description='Y axis Size:',
    disabled=False,
    continuous_update=False,
    orientation='Vertical',
    readout=True,
    readout_format='d'
)
figy.style.handle_color="lawngreen"

Scale = widgets.Dropdown(
    options=[('linear', 1), ('logarithmic', 2)],
    value=1,
    description='Scale',
)

Style = widgets.Dropdown(
    options=[('default', 'default'), ('classic', 'classic'),
             ('Solarize_Light2', 'Solarize_Light2'),('_classic_test_patch','_classic_test_patch'),
             ('bmh', 'bmh'),('dark_background', 'dark_background'),
             ('fast', 'fast'),('fivethirtyeight', 'fivethirtyeight'),
             ('ggplot', 'ggplot'),('grayscale', 'grayscale'),
             ('seaborn-colorblind', 'seaborn-colorblind'),('seaborn-dark', 'seaborn-dark'),
             ('seaborn-dark-palette', 'seaborn-dark-palette'),('seaborn-darkgrid', 'seaborn-darkgrid'),
             ('seaborn-deep', 'seaborn-deep'),('seaborn-muted', 'seaborn-muted'),
             ('seaborn-notebook', 'seaborn-notebook'),('seaborn-paper', 'seaborn-paper'),
             ('seaborn-pastel', 'seaborn-pastel'),('seaborn-poster', 'seaborn-poster'),
             ('seaborn-talk', 'seaborn-talk'),('seaborn-white', 'seaborn-white'),
             ('seaborn-whitegrid', 'seaborn-whitegrid'),('seaborn-colorblind10', 'seaborn-colorblind10')],
    value='default',
    description='Style',
)

DataToInclude = widgets.SelectMultiple(
    options=['East Surface', 'East Bottom', 'West'],
    value=['East Surface'],
    description='Datasets',
    rows = 3,
    disabled=False
)
vbox_g = widgets.VBox([Style,Scale,DataToInclude,figx])
hbox_g = widgets.HBox([figy,vbox_g])
vbox_general = widgets.VBox([Header,hbox_g])
############################################################################


#################### ENVIRONMENT ###########################################
EnvDataPoints = widgets.SelectMultiple(
    options=['Salinity_ppt', 'pH', 'DO_mg/L','Turb_NTU','Chl_ug/L','Temp_C'],
    value=['pH'],
    rows=6,
    description='Data',
    disabled=False
)

StartDate = widgets.DatePicker(description='Start Date')

EndDate = widgets.DatePicker(description='End Date')

dates = [datetime.date(2018,i,1) for i in range(1,13)]
options = [(i.strftime('%b'), i) for i in dates]
Months = widgets.SelectionRangeSlider(
    options=options,
    index=(0,11),
    description='Months',
    disabled=False
)


Years=widgets.SelectMultiple(
    options=['2018', '2019', '2020'],
    value=['2018', '2019', '2020'],
    description='Years',
    rows = 3,
    disabled=False
)

vbox_dateselecter =  widgets.VBox([StartDate,EndDate])

vbox_yearlimiter =  widgets.VBox([Months,Years])

DRtab = widgets.Tab()
DRtab.children = [vbox_dateselecter,vbox_yearlimiter]
DRtab.set_title(0,'Date Range')
DRtab.set_title(1,'Years Overlayed')

vbox_EnvData = widgets.VBox([EnvDataPoints,DRtab])
############################################################################


############## SPECIES #####################################################
species = ['Amphipod','Barnacle','Clam Worm','Colonial Ciliates','Copepod',
             'Dark False Mussel','Dusky Sea Slug','Hydra','Magnification',
             'Mud Crab','Oyster Flatworm','Stentor','Tube Bryzoan','Whip Mudworm']
SpecDataPoints = widgets.SelectMultiple(
    options=species,
    value=['Hydra'],
    rows=14,
    description='Data',
    disabled=False
)

vbox_SpecData =  widgets.VBox([SpecDataPoints,DRtab])
############################################################################


############## CORRELATION #################################################
EnvCDataPoints = widgets.Dropdown(
    options=envDatapoints,
    description='Data',
)

EnvC2DataPoints = widgets.Dropdown(
    options=envDatapoints,
    description='Data',
)

accordion = widgets.Accordion(children=[EnvCDataPoints,EnvC2DataPoints])
accordion.set_title(0, 'First Condition')
accordion.set_title(1, 'Second Condition')

SpecCDataPoints = widgets.Select(
    options=species,
    description='Data',
    orientation='Vertical'
)

EnvCDataPoints = widgets.SelectMultiple(
    options=['Salinity_ppt', 'pH', 'DO_mg/L','Turb_NTU','Chl_ug/L','Temp_C'],
    value=['pH'],
    rows=6,
    description='Correlate to:',
    disabled=False,
    max = 3
)
method = widgets.Dropdown(
    options={'Top': 0, 'Bottom': 1},
    value=0,
    description='',
    layout = Layout (width = '25%', height = '100%')
)
ResultNum=widgets.IntSlider(
    value=3,
    min=1,
    max=len(EnvCDataPoints.value),
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
ResultNum.style.handle_color="lawngreen"

def AdjMax(change):
    ResultNum.max=len(EnvCDataPoints.value)

EnvCDataPoints.observe(AdjMax, names='value')

vbox_env_env =  widgets.VBox([accordion])
hbox_env_spec_sub = widgets.HBox([method,ResultNum])
vbox_env_spec =  widgets.VBox([SpecCDataPoints,EnvCDataPoints,hbox_env_spec_sub])

CorrTab = widgets.Tab()

CorrTab.children = [vbox_env_env,vbox_env_spec]

CorrTab.set_title(0,'Environmental Correlation')
CorrTab.set_title(1,'Species Correlation')

vbox_Corr =  widgets.VBox([CorrTab])
############################################################################


############## MAIN ########################################################

MainTab = widgets.Tab()

MainTab.children = [vbox_EnvData,vbox_SpecData,vbox_Corr]

MainTab.set_title(0,'Environmental')
MainTab.set_title(1,'Species')
MainTab.set_title(2,'Correlation')
MainTab.layout = Layout (width = '400px', height = '100%')
vbox_main = widgets.VBox([MainTab])
###########################################################################


############ DISPLAY ######################################################
clear_output()
   
progress =  widgets.IntProgress(
    value=0,
    min=0,
    max=2,
    step=1,
    description='plotting:',
    style = {'bar_color':'lawngreen'},
    orientation='horizontal'
    )

btn_plt = widgets.Button(
    description='''Plot 📈''',
    tooltip='Plot',
    style={'button_color': 'lawngreen'},
    layout = Layout (width = '370px', height = '100%'),
    font_size=75
    #layout = Layout(width='98%', height='100%')
)

Output = widgets.Output()

def on_button_clicked(event):
    with Output:
        clear_output()
        print(f".{Months.value}.{DRtab.selected_index}")
    if DRtab.selected_index == 0:
        DPlot(DataToInclude.value,EnvDataPoints.value,
              Scale.value,StartDate.value,EndDate.value,
              figx.value,figy.value,Style.value)
    else:
        YPlot(DataToInclude.value,EnvDataPoints.value,
              Scale.value,Years.value,Months.value,
              figx.value,figy.value,Style.value)

btn_plt.on_click(on_button_clicked)

vbox_result = widgets.VBox([progress,btn_plt, Output])
vbox_Setup = widgets.VBox([vbox_general,vbox_result])
hbox_Setup = widgets.HBox([vbox_Setup,vbox_main])


In [ ]:
#page = widgets.HBox([vbox_Setup])
page = hbox_Setup
display(page)
display(plt.gca())

In [ ]:

# int_range = widgets.IntSlider()
# display(int_range)

# def on_value_change(change):
#     int_range.max = int_range.value*3+1-int_range.value^0

# int_range.observe(on_value_change, names='value')

In [ ]:
# 0^0